In this notebook, im going to write the script that is in charge of , based on values of pi and beta, be able to select the snps that are going to be contributing to the phenotype and calculate their effect sizes 
and then calculate the optima phenotypes based on a temperature gradient and the contributing snps
and then finally i will also generate the fasta file used as reference for slim, basically generated from the vcf file 

In [1]:
import numpy as np
import pandas as pd
import allel
import random

In [2]:
## params for annotating vcf file with sc
chr_number = 5
vcf_file = 'chr5_grenenet_subset.vcf'
pi =  0.01
beta = 4

#params for calculating optima 
grenenet_1001g_ecotypes = 'ecotypes_grenenet_1001g.txt'
path_worldclim_ecotypesdata = '/Users/tbellagio/safedata/ath_evo/grenephase1/data/worldclim_ecotypesdata.csv'
bed_sc = 'selection_coef_chr5_subset.bed'

## params for gen of fasta from vcf 
path_vcf = '' 
path_fasta = ''

pi can be interpreted as the probability of a bernoulli trial for each snps contributing to the qtl
because there are many snps, this can be thought as a binomial with n trials (number of positions in the chromosome) and each with probability pi. 
Becuase the expected value or mean of a binomial is n*p we can just multiply those values to get to the expected value of snps contributing to the qtl 

In [3]:
file_p = allel.read_vcf(vcf_file, fields='variants/POS')
## get all the pos from the vcf file
file_p = allel.read_vcf(vcf_file, fields='variants/POS')
pos = file_p['variants/POS']
n_pos = len(pos)
n_pos_contr =round(pi*n_pos)
## randomly select the positions that are going to contribute 
pos_contr = random.sample(pos.tolist(), k=n_pos_contr)
## now im gonna sample the number of nps contributing from a normal distribution with mean 0 and sd defined by beta 
mu, sigma = 0,beta # mean and standard deviation
effect_sizes = np.random.normal(mu, sigma, n_pos_contr)
effect_sizes = np.round(effect_sizes, 4)
## now i have to create a bed file to add this infromation to the vcf file 
## create dataframe of the contributing pos 


## to caluclate the positions FROM in a bed file, the fist value is not included
positions_from = [i-1 for i in pos]
# Define the positions and values
chromosome = [chr_number] * len(pos)

# Create a DataFrame from the positions and values
all_pos = pd.DataFrame({'chromosome':chromosome,'positions_from': positions_from, 'positions_to': pos})
contrib_pos = pd.DataFrame({'positions_to': pos_contr, 'sel_coef':effect_sizes})

# merge and fillna with 0 since that is the selection coefficient for all the non contributing snps 
all_pos = all_pos.merge(contrib_pos, left_on= 'positions_to', right_on= 'positions_to' ,how='left').fillna(0)



In [4]:
# Write the DataFrame to a BED file
all_pos.to_csv(bed_sc, sep='\t', header=False, index=False)

In [5]:
## optima caclculation

In [6]:
ecotypes_grenenet_1001g = pd.read_csv(grenenet_1001g_ecotypes).columns.astype(int)
ecotypes_temp = pd.read_csv(path_worldclim_ecotypesdata, usecols=['ecotypeid', 'bio1'])
#BIO1 = Annual Mean Temperature
## filter by the ones coming from the 1001 genomes
ecotypes_temp = ecotypes_temp[ecotypes_temp['ecotypeid'].isin(ecotypes_grenenet_1001g)]
## get the total number of position in the vcf file that i am working with, so then i can calculate the selection coefficients 
max_t = ecotypes_temp['bio1'].max()
min_t = ecotypes_temp['bio1'].min()

samples = pd.DataFrame(columns = ecotypes_temp.columns)

num_ranges = 5

# calculate the range width
range_width = (max_t - min_t) / num_ranges

# create the ranges
ranges = [(min_t + i*range_width, min_t + (i+1)*range_width) for i in range(num_ranges)]

# sample across the ranges
for r in ranges:
    r_ages = ecotypes_temp[( ecotypes_temp['bio1'] >= r[0]) & ( ecotypes_temp['bio1'] <= 1+r[1])]
    samples = pd.concat([samples, r_ages.sample(7, replace=True)])
## and then to complete i will drop one randomly 
drop_index = np.random.choice(samples.index)
samples = samples.drop(drop_index)

selection_coef = pd.read_csv(bed_sc, sep = '\t',header = None)[3]

vcf = allel.read_vcf(vcf_file)
phenotypes = []
for ecotype in samples['ecotypeid']:
    vcf_ecotype = allel.read_vcf(vcf_file, samples=[str(ecotype)])
    ## for each position this is the number of alterantive variants 
    alt_alleles_per_pos = vcf_ecotype['calldata/GT'].sum(axis=2)
    gen_effectsize = np.multiply(alt_alleles_per_pos.flatten(), np.array(selection_coef))  ## select coef are actually effect sizes
    phenotypes.append(gen_effectsize.sum())
samples['phenotype'] = phenotypes
samples = samples.reset_index(drop=True)


In [7]:
# write optimas
for i in range(0,34):
    optima = pd.concat([samples[samples.index == i]]*12, ignore_index=True)
    optima.to_csv(f'optima_files/optima{i}.csv')
    optima['phenotype'].to_csv(f'optima_files/optima{i}_slim.txt', header=None, sep='\n', index=None)

In [8]:
### generation of reference fasta from vcf 

In [9]:
#def from_vcf_to_fasta(vcf_file_name, path_vcf, path_fasta):
vcf = allel.read_vcf(path_vcf + vcf_file)
seq_firstchr = ''
for i in range(0, len(vcf['variants/REF'])):
    seq_firstchr += vcf['variants/REF'][i]

start = vcf['variants/POS'][0]
end = vcf['variants/POS'][-1]
chro = vcf['variants/CHROM'][0]
with open(path_fasta + vcf_file[:-4] + '.fasta', 'a') as f:
    f.write(f'>{chro}:{start}-{end}\n')
    f.write(seq_firstchr + '\n')